In [7]:
# converter o database fake em sqlite:

import json
import sqlite3

with open('alunos.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

conn = sqlite3.connect('dados_alunos.db')
c = conn.cursor()

c.execute("CREATE TABLE IF NOT EXISTS alunos (Nome TEXT, Serie TEXT, Turma TEXT, Matricula TEXT);")

for student in data['students']:
    c.execute('''
        INSERT INTO alunos (Nome, Serie, Turma, Matricula)
        VALUES (?, ?, ?, ?);
    ''', (
        student['Nome'],
        student['Serie'],
        student['turma'],
        int(student['Matricula'])
    ))

conn.commit()
conn.close()



In [22]:
#dar uma olhada se deu bom:

import pandas as pd

conn = sqlite3.connect('dados_alunos.db')

df_total = pd.read_sql_query("SELECT * FROM alunos", conn)

print(df_total.head(5))

conn.close()

                       Nome Serie Turma Matricula
0          Ana Paula Arósio    1ª     A  12345678
1  Angélica que vai de Táxi    2ª     B  87654321
2       Monalisa da Pintura    1ª     D  98765432
3              Renato Russo    2ª     A  34567890
4              Felipe Costa    3ª     B  87654310


In [19]:
## Script pra excluir as matrículas duplicadas no database fake

conn = sqlite3.connect('dados_alunos.db')
c = conn.cursor()

c.execute('''
    SELECT Matricula
    FROM alunos
    GROUP BY Matricula
    HAVING COUNT(Matricula) > 1;
''')

results = c.fetchall()

matriculas_duplicadas = [row[0] for row in results]

for matricula in matriculas_duplicadas:
    c.execute('''
        DELETE FROM alunos
        WHERE Matricula = ?;
    ''', (matricula,))
conn.commit()
conn.close()